In [7]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [3]:
data = pd.read_excel("//ru.nl/wrkgrp/TeamIR/Man_info/Student Analytics/Prognosemodel RU/Syntax/Python/studentprognose/data/output/output_final.xlsx")
print(data)
print(data.columns)

         index Examentype                                Croho groepeernaam  \
0            0   Bachelor                   B Algemene Cultuurwetenschappen   
1            1   Bachelor                   B Algemene Cultuurwetenschappen   
2            2   Bachelor                   B Algemene Cultuurwetenschappen   
3            3   Bachelor                   B Algemene Cultuurwetenschappen   
4            4   Bachelor                   B Algemene Cultuurwetenschappen   
...        ...        ...                                               ...   
164780  163157     Master  M Wijsbegeerte van een Bepaald Wetenschapsgebied   
164781  163158     Master  M Wijsbegeerte van een Bepaald Wetenschapsgebied   
164782  163159     Master  M Wijsbegeerte van een Bepaald Wetenschapsgebied   
164783  163160     Master  M Wijsbegeerte van een Bepaald Wetenschapsgebied   
164784  163161     Master  M Wijsbegeerte van een Bepaald Wetenschapsgebied   

       Herkomst Faculteit  Collegejaar  Weeknummer 

In [16]:
percentage_range = range(10, 150, 10)

In [4]:
import json

def print_dict(dict):
    print(json.dumps(dict, sort_keys=True, indent=4))

In [5]:
def calculate_weight_distribution(metrics, percentage, second_percentage):
    sorted_metrics = sorted(metrics.items(), key=lambda x:x[1])
    sorted_metrics = dict(sorted_metrics)

    keys = list(sorted_metrics.keys())

    first_threshold = 1.0 + (float(percentage) / 100)
    second_threshold = 1.0 + (float(second_percentage) / 100)

    # Ignore second lowest if it is above threshold
    if sorted_metrics[keys[1]] > sorted_metrics[keys[0]] * first_threshold:
        return {keys[0]: 1.0, keys[1]: 0.0, keys[2]: 0.0}
    elif sorted_metrics[keys[2]] > sorted_metrics[keys[1]] * second_threshold:
        total = sorted_metrics[keys[0]] + sorted_metrics[keys[1]]
        new_total = total / sorted_metrics[keys[0]] + total / sorted_metrics[keys[1]]

        return {keys[0]: (total / sorted_metrics[keys[0]]) / new_total, keys[1]: (total / sorted_metrics[keys[1]]) / new_total, keys[2]: 0.0}
    else:
        total = sorted_metrics[keys[0]] + sorted_metrics[keys[1]] + sorted_metrics[keys[2]]
        new_total = total / sorted_metrics[keys[0]] + total / sorted_metrics[keys[1]] + total / sorted_metrics[keys[2]]

        return {keys[0]: (total / sorted_metrics[keys[0]]) / new_total, keys[1]: (total / sorted_metrics[keys[1]]) / new_total, keys[2]: (total / sorted_metrics[keys[2]]) / new_total}

def get_metric_weight_distribution(data, programme, herkomst, percentage, second_percentage):
    data = data[data["Croho groepeernaam"] == programme]
    data = data[data["Herkomst"] == herkomst]

    MAE_results = {"c": np.nanmean(data["MAE sarima cumulative"]), "i": np.nanmean(data["MAE sarima individual"]), "r": np.nanmean(data["MAE ratio"])}
    MAE_results = calculate_weight_distribution(MAE_results, percentage, second_percentage)

    MAPE_results = {"c": np.nansum(data["MAPE sarima cumulative"]) / len(data) * 100, "i": np.nansum(data["MAPE sarima individual"]) / len(data) * 100,
                    "r": np.nansum(data["MAPE ratio"]) / len(data) * 100}
    MAPE_results = calculate_weight_distribution(MAPE_results, percentage, second_percentage)

    return MAE_results, MAPE_results

In [8]:
weight_distribution = {"Programme": [], "Herkomst": [], "MAE c": [], "MAE i": [], "MAE r": [], "MAPE c": [], "MAPE i": [], "MAPE r": [], "Percentage": [], "Second percentage": []}

for percentage in percentage_range:
    for second_percentage in percentage_range:
        for programme in data["Croho groepeernaam"].unique():
            for herkomst in data["Herkomst"].unique():
                weight_distribution["Programme"].append(programme)
                weight_distribution["Herkomst"].append(herkomst)
                MAE_results, MAPE_results = get_metric_weight_distribution(data, programme, herkomst, percentage, second_percentage)
                weight_distribution["MAE c"].append(MAE_results["c"])
                weight_distribution["MAE i"].append(MAE_results["i"])
                weight_distribution["MAE r"].append(MAE_results["r"])
                weight_distribution["MAPE c"].append(MAPE_results["c"])
                weight_distribution["MAPE i"].append(MAPE_results["i"])
                weight_distribution["MAPE r"].append(MAPE_results["r"])
                weight_distribution["Percentage"].append(percentage)
                weight_distribution["Second percentage"].append(second_percentage)

weight_distribution_data = pd.DataFrame(weight_distribution)
weight_distribution_data.to_excel("../configuration/weight_distribution.xlsx", index=False)

C:\Users\jjble\AppData\Local\Temp\ipykernel_18608\2073443432.py:28: RuntimeWarning: Mean of empty slice
  MAE_results = {"c": np.nanmean(data["MAE sarima cumulative"]), "i": np.nanmean(data["MAE sarima individual"]), "r": np.nanmean(data["MAE ratio"])}
C:\Users\jjble\AppData\Local\Temp\ipykernel_18608\2073443432.py:20: RuntimeWarning: invalid value encountered in scalar divide
  new_total = total / sorted_metrics[keys[0]] + total / sorted_metrics[keys[1]] + total / sorted_metrics[keys[2]]
C:\Users\jjble\AppData\Local\Temp\ipykernel_18608\2073443432.py:22: RuntimeWarning: invalid value encountered in scalar divide
  return {keys[0]: (total / sorted_metrics[keys[0]]) / new_total, keys[1]: (total / sorted_metrics[keys[1]]) / new_total, keys[2]: (total / sorted_metrics[keys[2]]) / new_total}
C:\Users\jjble\AppData\Local\Temp\ipykernel_18608\2073443432.py:31: RuntimeWarning: invalid value encountered in scalar divide
  MAPE_results = {"c": np.nansum(data["MAPE sarima cumulative"]) / len(dat

In [10]:
def mean_absolute_error(true, prediction):
    return abs(true - prediction)

def mean_absolute_percentage_error(true, prediction):
    return abs((true - prediction) / true)

In [17]:
weight_data = pd.read_excel("../configuration/weight_distribution.xlsx")

error_rates = {"Percentage": [], "Second percentage": [], "MAE": [], "MAPE": [], "Herkomst": [], "MAE of AE": [], "MAPE of AE": []}
for herkomst in ["EER", "NL", "Niet-EER"]:
    for percentage in percentage_range:
        for second_percentage in percentage_range:
            error_rates["Percentage"].append(percentage)
            error_rates["Second percentage"].append(second_percentage)

            MAE_total = 0.0
            MAPE_total = 0.0

            MAE_total_AE = 0.0
            MAPE_total_AE = 0.0

            filtered_weight_data = weight_data[weight_data["Percentage"] == percentage]
            filtered_weight_data = filtered_weight_data[filtered_weight_data["Second percentage"] == second_percentage]
            filtered_weight_data = filtered_weight_data[filtered_weight_data["Herkomst"] == herkomst]
            for i, row_weight in filtered_weight_data.iterrows():
                filtered_data = data[data["Croho groepeernaam"] == row_weight["Programme"]]
                filtered_data = filtered_data[filtered_data["Herkomst"] == row_weight["Herkomst"]]
                filtered_data = filtered_data[filtered_data["Collegejaar"] >= 2021]

                use_average_ensemble = False
                if row_weight["MAE c"] + row_weight["MAE i"] + row_weight["MAE r"] != 1:
                    use_average_ensemble = True
                    
                MAE_subtotal = 0.0
                MAPE_subtotal = 0.0

                MAE_subtotal_AE = 0.0
                MAPE_subtotal_AE = 0.0
                for j, row in filtered_data.iterrows():
                    if use_average_ensemble:
                        if not np.isnan(row["MAE average ensemble"]):
                            MAE_subtotal += row["MAE average ensemble"]
                        if not np.isnan(row["MAPE average ensemble"]):
                            MAPE_total += row["MAPE average ensemble"]
                    else:
                        if not np.isnan(row["SARIMA_cumulative"]) and not np.isnan(row["SARIMA_individual"]) and not np.isnan(row["Prognose_ratio"]) and not np.isnan(row["Aantal_studenten"]):
                            new_ensemble_prediction = row["SARIMA_cumulative"] * row_weight["MAE c"] + row["SARIMA_individual"] * row_weight["MAE i"] + row["Prognose_ratio"] * row_weight["MAE r"]
                            MAE_subtotal += mean_absolute_error(row["Aantal_studenten"], new_ensemble_prediction)
                            MAPE_subtotal += mean_absolute_percentage_error(row["Aantal_studenten"], new_ensemble_prediction)

                    if not np.isnan(row["MAE average ensemble"]):
                        MAE_subtotal_AE += row["MAE average ensemble"]
                    if not np.isnan(row["MAPE average ensemble"]):
                        MAPE_subtotal_AE += row["MAPE average ensemble"]

                count = len(filtered_data)
                if count == 0:
                    count = 1

                MAE_total += MAE_subtotal / count
                MAPE_total += MAPE_subtotal / count * 100

                MAE_total_AE += MAE_subtotal_AE / count
                MAPE_total_AE += MAPE_subtotal_AE / count * 100

            error_rates["MAE"].append(MAE_total / len(filtered_weight_data))
            error_rates["MAPE"].append(MAPE_total / len(filtered_weight_data))
            error_rates["Herkomst"].append(herkomst)
            error_rates["MAE of AE"].append(MAE_total_AE / len(filtered_weight_data))
            error_rates["MAPE of AE"].append(MAPE_total_AE / len(filtered_weight_data))

error_rates = pd.DataFrame(error_rates)
error_rates.to_excel("../configuration/error_rates.xlsx", index=False)

In [18]:
weight_data = pd.read_excel("../configuration/weight_distribution.xlsx")
error_rates = pd.read_excel("../configuration/error_rates.xlsx")

percentage_per_herkomst = {"EER": (), "NL": (), "Niet-EER": ()}
for herkomst in (percentage_per_herkomst.keys()):
    temp_data = error_rates[error_rates["Herkomst"] == herkomst]
    temp_data = temp_data[temp_data["MAE"] == temp_data["MAE"].min()]
    percentage_per_herkomst[herkomst] = (float(temp_data["Percentage"].iloc[0]), float(temp_data["Second percentage"].iloc[0]))

print(percentage_per_herkomst)

ensemble_weights = {"Programme": [], "Herkomst": [], "SARIMA_cumulative": [], "SARIMA_individual": [], "Prognose_ratio": [], "Average_ensemble_prediction": []}
for herkomst in (percentage_per_herkomst.keys()):
    temp_data = weight_data[weight_data["Herkomst"] == herkomst]
    temp_data = temp_data[temp_data["Percentage"] == percentage_per_herkomst[herkomst][0]]
    temp_data = temp_data[temp_data["Second percentage"] == percentage_per_herkomst[herkomst][1]]

    for i, row in temp_data.iterrows():
        ensemble_weights["Programme"].append(row["Programme"])
        ensemble_weights["Herkomst"].append(row["Herkomst"])

        average_ensemble_weight = 0.0
        MAE_c = row["MAE c"]
        MAE_i = row["MAE i"]
        MAE_r = row["MAE r"]
        if MAE_c + MAE_i + MAE_r != 1 or MAE_c + MAE_i + MAE_r == 0:
            average_ensemble_weight = 1.0
            MAE_c = 0.0
            MAE_i = 0.0
            MAE_r = 0.0

        ensemble_weights["SARIMA_cumulative"].append(MAE_c)
        ensemble_weights["SARIMA_individual"].append(MAE_i)
        ensemble_weights["Prognose_ratio"].append(MAE_r)

        ensemble_weights["Average_ensemble_prediction"].append(average_ensemble_weight)

ensemble_weights = pd.DataFrame(ensemble_weights)
ensemble_weights.to_excel("../configuration/ensemble_weights.xlsx", index=False)

{'EER': (70.0, 70.0), 'NL': (70.0, 120.0), 'Niet-EER': (40.0, 30.0)}
